In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.types._

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)
    .withColumn("timestamp", lit(current_timestamp()))

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string, timestamp: timestamp]


[tweet: string, timestamp: timestamp]

In [4]:
val model = PipelineModel.load("/home/jovyan/models/spark-ml-model")

model = pipeline_3dfeba337f5a


pipeline_3dfeba337f5a

In [5]:
val predictionsDF = model.transform(inputDF)

predictionsDF = [tweet: string, timestamp: timestamp ... 6 more fields]


[tweet: string, timestamp: timestamp ... 6 more fields]

In [6]:
import org.apache.spark.sql.functions._
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [7]:
import spark.implicits._

Name: Compile Error
Message: <console>:39: error: stable identifier required, but this.$line7$read.spark.implicits found.
       import spark.implicits._
                    ^

StackTrace: 

In [10]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.persist()
    
    val cleanProbability = batchDF.select(getProbability($"probability").alias("clean_probability"))
    cleanProbability.write.mode("append").format("console").save()
    
    val windowCount = batchDF
        .withWatermark("timestamp", "20 seconds")
        .groupBy(window($"timestamp", "10 seconds", "1 seconds"), $"prediction")
        .count()
    windowCount.write.mode("append").format("console").save()
    
    batchDF.unpersist()
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@7c45144f

+-------------------+
|  clean_probability|
+-------------------+
|  0.506119085226794|
|0.44409918165014944|
|0.49065237891017643|
| 0.5115763550216391|
| 0.5021846446881393|
|0.48400688875371084|
|  0.506119085226794|
| 0.5095230413582319|
| 0.4984422398503117|
| 0.5115763550216391|
| 0.5594890772238319|
| 0.5115763550216391|
|0.44409918165014944|
| 0.5115763550216391|
+-------------------+

